In [2]:
import sys
import io
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set UTF-8 encoding for stdout
# Image data generators for loading and augmenting images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,  # Use 20% of data for validation
    fill_mode='nearest'
)

# Correct path to your dataset directory
dataset_dir = 'D:\Coursat\NTI\HCIA AI\Proje\Driver Drowsiness Dataset (DDD)'

# Train generator
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# Validation generator
val_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Load the VGG16 model pre-trained on ImageNet, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base to retain pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for drowsiness detection
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification: 'drowsy' or 'non-drowsy'

# Define the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model with Adam optimizer and binary cross-entropy loss
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define callbacks for early stopping and model checkpointing
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('drowsiness_detection_vgg16.keras', save_best_only=True)

# Callbacks list
callbacks = [early_stopping, model_checkpoint]

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Coursat\\NTI\\HCIA AI\\Proje\\Driver Drowsiness Dataset (DDD)'

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=2,
    validation_data=val_generator,
    callbacks=callbacks
)

In [ ]:

# Plot training & validation accuracy values
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

In [ ]:
# Save the trained model
model.save('drowsiness_detection_vgg16_final.h5')